Vajalikud impordid

In [1]:
import pandas as pd
import re
import string
import nltk
import tensorflow as tf
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Andmete sisse lugemine

In [2]:
train = pd.read_csv('train.csv')
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [3]:
test = pd.read_csv('test.csv')
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [4]:
'''Loeme kokku NaN väärtusi igas veerus'''

for col in test.columns:
    print(col, test[col].isna().sum())

id 0
keyword 26
location 1105
text 0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Eeltöötlus

Eeltöötluse käigus puhastasin andmestiku, suuremas osas just "text" veergu. Kasutasin selle jaoks regexit.

In [6]:
'''Jätame alles ainult text-nimelise veeru, kuna ülejäänud veerud on minu arvates ennustamiseks ebavajalikud'''

train = train.drop(['id', 'keyword', 'location'], axis=1) 
test = test.drop(['keyword', 'location'], axis=1)
train

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


In [7]:
'''Defineerime funktsioonid teksti töötlemiseks ja puhastamiseks'''

def remove_punctuation(text):
    # ['!','"','$','%','&',"'",'(',')','*',
    # '+',',','-','.','/',':',';','<','=',
    # '>','?','@','[','\\',']','^','_','`',
    # '{','|','}','~']
    # kustutab tekstist ülaltoodud märgid
    punctuations = list(string.punctuation)
    table = str.maketrans('', '', ''.join(punctuations))
    return text.translate(table)

def remove_tag(text): 
    # kustutab @ märgi
    tag = re.compile(r'@\S+')
    return tag.sub(r'',text)

def remove_URL(text):
    # kustutab kõik URL
    url = re.compile(r'https?://\S+|www\.\S+')
    return re.sub(url,'',text)

In [8]:
'''nltk kustutab nn stopp-sõnad (näiteks "the" inglise sõnastikus)'''

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
'''Rakendame üleval defineeritud funktsioonid'''

train['cleaned'] = train['text'].apply(lambda x:remove_punctuation(x)).apply(lambda x:remove_tag(x)).apply(lambda x:remove_URL(x)).apply(lambda x: x.lower()).apply(lambda x: word_tokenize(x)).apply(lambda x:remove_stopwords(x))
test['cleaned'] = test['text'].apply(lambda x:remove_punctuation(x)).apply(lambda x:remove_tag(x)).apply(lambda x:remove_URL(x)).apply(lambda x: x.lower()).apply(lambda x: word_tokenize(x)).apply(lambda x:remove_stopwords(x))

Tokenizing

Teksti närvivõrkudesse "söötimiseks" peame teksti tokeniseerima ja seejärel indekseerima või vektoriseerima. Siin me muudame iga teksti indeksite jadaks, kasutades tensorflow.keras.preprocessing.text.Tokenenizer, kus igal ainulaadsel sõnal on oma register.

In [10]:
VOCAB_SIZE = 10000
MAX_LENGTH = 20

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>", filters="")
tokenizer.fit_on_texts(train['text'])

train_seq = tokenizer.texts_to_sequences(train['text'])
train_padded = pad_sequences(train_seq, maxlen=MAX_LENGTH)

test_seq = tokenizer.texts_to_sequences(test['text'])
test_padded = pad_sequences(test_seq, maxlen=MAX_LENGTH)

Mudel

In [11]:
'''Defineerime mudelit hüperparameetritega'''

model = tf.keras.Sequential([
        tf.keras.layers.Embedding(VOCAB_SIZE, output_dim=16, input_length=MAX_LENGTH),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16, dropout=0.4)),
        tf.keras.layers.Dense(1, activation="sigmoid"),
])
model.compile(optimizer= "adam", loss= "binary_crossentropy", metrics="accuracy")

In [12]:
hist = model.fit(x=train_padded,y=train['target'], epochs=50, batch_size=32)

Epoch 1/50
238/238 [==============================] - 5s 8ms/step - loss: 0.6292 - accuracy: 0.6272
Epoch 2/50
238/238 [==============================] - 2s 8ms/step - loss: 0.4021 - accuracy: 0.8225
Epoch 3/50
238/238 [==============================] - 2s 8ms/step - loss: 0.3034 - accuracy: 0.8757
Epoch 4/50
238/238 [==============================] - 2s 8ms/step - loss: 0.2419 - accuracy: 0.9041
Epoch 5/50
238/238 [==============================] - 2s 8ms/step - loss: 0.2007 - accuracy: 0.9217
Epoch 6/50
238/238 [==============================] - 2s 8ms/step - loss: 0.1797 - accuracy: 0.9303
Epoch 7/50
238/238 [==============================] - 2s 8ms/step - loss: 0.1533 - accuracy: 0.9414
Epoch 8/50
238/238 [==============================] - 2s 8ms/step - loss: 0.1361 - accuracy: 0.9510
Epoch 9/50
238/238 [==============================] - 2s 8ms/step - loss: 0.1265 - accuracy: 0.9527
Epoch 10/50
238/238 [==============================] - 2s 8ms/step - loss: 0.1152 - accuracy: 0.9603

In [13]:
print("Mudeli täpsus on", hist.history["accuracy"][-1])

Mudeli täpsus on 0.9813476800918579


In [14]:
'''Kirjutame tulemused faili'''

predictions = pd.DataFrame({'target': (model.predict(test_padded, verbose=0) > 0.5).reshape(-1).astype(int)}, index=test.id)
predictions.index.name='id'
with open('sample_submission.csv', 'w') as f:
    predictions.to_csv(f, line_terminator='\n')